In [139]:
from pathlib import Path
from typing import List

import dspy
from pydantic import BaseModel, Field

lm = dspy.LM('openai/gpt-4o-mini')
# lm = dspy.LM('ollama_chat/qwen2.5:14b')#, api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

In [140]:
# lm("Say this is a test!")#, temperature=0.7)  # => ['This is a test!']

In [141]:
# qa = dspy.ChainOfThought('question -> answer')
# qa(question="What is the meaning of life?")

In [142]:
# content = """# Project Guidelines
#
# ## Code Style
#
# - Use 4 spaces for indentation
# - Maximum line length is 100 characters
# - Use camelCase for variable names
# - Use PascalCase for class names
# - Use snake_case for function names
# """

cwd = Path.cwd()
print(cwd)
with open(Path.cwd() / ".." / ".." / ".github" / "copilot-instructions.md", "r") as f:
    content = f.read()

/Users/svetzal/Work/Personal/ml/context-mixer/src/_examples


In [143]:
class AtomicIdea(BaseModel):
    content: str = Field(..., description="The content of the atomic idea")
    context: str = Field(..., description="The context in which the idea was defined or in which it is valid")
    confidence: float = Field(..., description="Confidence in context detection (0.0-1.0)", ge=0.0, le=1.0)

class ExtractAtomicIdeas(dspy.Signature):
    """The content provided is a detailed and potentially lengthy document containing many discreet atomic ideas. Extract every distinct detailed atomic idea from it, along with the context in which the idea was defined or in which it is valid."""
    content: str = dspy.InputField()

    ideas: List[AtomicIdea] = dspy.OutputField()

In [144]:
splitter = dspy.ChainOfThought(ExtractAtomicIdeas)
result = splitter(content=content)
result

Prediction(
    reasoning='The provided content outlines comprehensive guidelines for the development of a command-line tool called Context Mixer, which is aimed at enhancing the usability of GenAI coding assistants. It includes sections on project overview, identity and branding, engineering principles, tech stack, project structure, development setup, coding guidelines, testing guidelines, documentation, LLM tool development, procedural guidelines, and release processes. Each section contains distinct atomic ideas that can be extracted, such as specific coding principles, project structure details, and testing best practices. These ideas are relevant in the context of software development, particularly for projects involving AI and command-line tools.',
    ideas=[AtomicIdea(content='Provide a single command-line tool that helps developers create, organise, merge and deploy reusable prompt instructions across multiple GenAI coding assistants.', context='Project Overview', confidence=